# PSyclone tutorial: NEMO API Example 2 - Unsupported Code

The PSyIR is designed to capture the computational parts of a code (as these are what we want to optimise and parallelise). This example shows how PSyclone deals with any unsupported code in the NEMO API.

First let's specify a simple Fortran code in a Python string. This example is the same as in Example 1 except that it includes a call to another subroutine, an allocate and a write to a file. The last two are added because the PSyIR does not currently capture either of these constructs.

In [ ]:
code = '''program test
  implicit none
  integer, parameter :: jpi=10, jpj=10, jpk=10
  real, allocatable, dimension(:,:,:) :: a,b
  integer :: ji,jj,jk
  allocate(a(jpi,jpj,jpk), b(jpi,jpj,jpk))
  call timer_start()
  do jk=1,jpk
    do jj=1,jpj
      do ji=1,jpi
        b(ji,jj,jk) = 0.0
      end do
    end do
  end do
  do jk=1,jpk
    do jj=1,jpj
      do ji=1,jpi
        a(ji,jj,jk) = b(ji,jj,jk)
      end do
    end do
  end do
  call timer_end()
  write (6,*) "HELLO"
end program test'''

Next we parse the code, create the PSyIR and take a look at it.

In [ ]:
from fparser.common.readfortran import FortranStringReader
reader = FortranStringReader(code)
from fparser.two.parser import ParserFactory
parser = ParserFactory().create(std="f2003")
parse_tree = parser(reader)

from psyclone.psyGen import PSyFactory
psy = PSyFactory("nemo").create(parse_tree)

invoke = psy.invokes.invoke_list[0]
schedule = invoke.schedule

schedule.view()

As can be seen, the write and allocate statements are captured in nodes called CodeBlocks. A CodeBlock is a sequence of statements that the PSyIR does not deal with and can therefore not transform/optimise.

This approach allows transformations/optimisations on the code that is supported whilst not requiring that the full Fortran code with its various esoteric features be captured. This, of course, assumes that all of the relevant computational code is captured in the PSyIR.

One way to think about this is that the PSyIR illuminates the computational parts of a code leaving the rest in the dark.

As the CodeBlock keeps the fparser2 representation of the code, this can be used to output the unmodified section of code when writing the output.

One limitation of this approach is that the parts of the code that the PSyIR does not understand must be output as Fortran code. Therefore, if a CodeBlock exists in the PSyIR representation of a Fortran program, the code must also be Fortran. It might be possible to create language wrappers around the CodeBlocks but this is not done at present.

Just to confirm that this approach works, let's write out the unmodified code.

In [ ]:
print((psy.gen))

Let's move on to the [next example](nemo_example3.ipynb)...